In [1]:

import torch
from transformers import AutoTokenizer, AutoModel

In [2]:
# tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
# model = AutoModel.from_pretrained('bert-base-uncased')

tokenizer = AutoTokenizer.from_pretrained('model_outputs')
model = AutoModel.from_pretrained('model_outputs')

Some weights of the model checkpoint at model_outputs were not used when initializing BertModel: ['mlp.dense.weight', 'mlp.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
sentences = [
    "chocolates are my favourite items.",
    "The fish dreamed of escaping the fishbowl and into the toilet where he saw his friend go.",
    "The person box was packed with jelly many dozens of months later.",
    "white chocolates and dark chocolates are favourites for many people.",
    "I love chocolates"
]

In [4]:
encoding = tokenizer.batch_encode_plus(sentences, add_special_tokens = True, truncation = True, padding = "max_length", return_attention_mask = True, return_tensors = "pt")


In [5]:
attention_mask = encoding['attention_mask']


In [6]:
#outputs[0] has the last hidden_state and outputs[1] has the pooler_output
outputs = model(**encoding)


In [7]:
# we are interested in the embedddings , i.e. the last hidden state 
embeddings = outputs[0] 
embeddings.size()

torch.Size([5, 512, 768])

In [8]:
#let us ge tthe attention_mask from our encoding.
attention_mask = encoding['attention_mask']


In [9]:
mask = attention_mask.unsqueeze(-1).expand(embeddings.size()).float()
mask.shape

torch.Size([5, 512, 768])

In [10]:
mask

tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 

In [11]:
masked_embeddings = embeddings * mask
masked_embeddings.shape

torch.Size([5, 512, 768])

In [12]:
summed = torch.sum(masked_embeddings, 1)
summed.shape

torch.Size([5, 768])

In [13]:
summed_mask = torch.clamp(mask.sum(1), min=1e-9)
summed_mask.shape

torch.Size([5, 768])

In [23]:
mean_pooled = summed / summed_mask
mean_pooled.shape

torch.Size([5, 768])

In [22]:
mean_pooled

array([[ 0.17034309,  0.20251192,  0.23745306, ...,  0.22712265,
         0.14471538, -0.40744543],
       [-0.04194761, -0.06746969, -0.05207163, ..., -0.08667774,
        -0.09079745,  0.25957498],
       [-0.27733576,  0.0870839 ,  0.11477065, ..., -0.27665672,
         0.13201816, -0.144713  ],
       [-0.17443156,  0.30543002,  0.4368227 , ..., -0.24344482,
        -0.12517875, -0.13542539],
       [ 0.2750279 ,  0.67376274,  0.06797466, ...,  0.12321696,
         0.24830884, -0.06427091]], dtype=float32)

In [24]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
# convert from PyTorch tensor to numpy array
mean_pooled = mean_pooled.detach().numpy()

# calculate
cosine_similarity(
    [mean_pooled[0]],
    mean_pooled[1:]
)

array([[0.48729715, 0.44618183, 0.79342073, 0.85763305]], dtype=float32)

In [25]:
# visualize the results
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
%matplotlib inline

# convert to numpy array
mean_pooled = mean_pooled.detach().numpy()

# calculate the cosine similarity
cosine_similarities = cosine_similarity(
    [mean_pooled[0]],
    mean_pooled[1:]
)

# plot the results
plt.figure(figsize=(10, 5))
sns.barplot(x=sentences[1:], y=cosine_similarities[0])
plt.xticks(rotation=90)
plt.show()




AttributeError: 'numpy.ndarray' object has no attribute 'detach'